# Extract ferredoxins and P450 from clusters annotated by antismash 5.0

In [1]:
import Bio
import re
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio import Entrez
import os
import glob

#fill in correct filenames for input/output files
path = 'Output/'
filenameout="tablep450ferredox.csv"
output="ferredox.fasta"
output1="p450ferre.fasta"

#creates empty data frame
tableofproteins=pd.DataFrame(columns=["accession",'p450', 'ferredoxin'])
listp450=[]
listferredox=[]

#process antismash file
for gb_file in glob.glob(os.path.join(path, '*.gbk')):
    
        gb_record = SeqIO.read(open(gb_file,"r"), "genbank")
        p450=[]
        ferredox=[]
        for feature in gb_record.features:
            if feature.type=="CDS":
                try:
                    if "ferredoxin" in feature.qualifiers['product'][0]:
                        pid=feature.qualifiers['protein_id'][0]
                        transl=feature.qualifiers['translation'][0]
                        listferredox.append(SeqRecord(Seq(transl),id=pid))
                        ferredox.append(pid)
                    if "P450" in feature.qualifiers['product'][0]:
                        pid=feature.qualifiers['protein_id'][0]
                        transl=feature.qualifiers['translation'][0]
                        listp450.append(SeqRecord(Seq(transl),id=pid))
                        p450.append(pid)
                except: pass
        new_row={"accession":gb_record.name,"p450":str(p450),"ferredoxin":str(ferredox)}
        tableofproteins = tableofproteins.append(new_row, ignore_index=True)
                    
#safe files
SeqIO.write(listp450, output1, 'fasta')
SeqIO.write(listferredox, output, 'fasta')
tableofproteins.to_csv(filenameout, index=False) 

# Get ferredoxins associated with P450s from NCBI files (fasta)

In [ ]:
import re
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
from Bio.SeqRecord import SeqRecord

filename_input="Fasta/ferredoxinncbi.fasta"
filename_output=filename_input+"info.csv"
outputferre=filename_input+"ferre.fasta"
outputp450=filename_input+"p450.fasta"
tableofproteins=pd.DataFrame(columns=['Protein Reference','Acessiongenome','Species',"Definition","proteins","blasts","products"])
Entrez.email = "friederike@biermann-erfurt.de"
listfasta=[]
liststrains=[]

def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def find_infrontof( s, last ):
    try:
        start = 0
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def find_after( s, first ):
    try:
        start =s.index( first ) + len( first )
        end = len(s)
        return s[start:end]
    except ValueError:
        return ""

listp450=[]
listferredoxins=[]
countsafe=0

#actually finding in fastas
for record in SeqIO.parse(filename_input, "fasta"):
    countsafe=countsafe+1
    try:
            if "ref" in record.id:
                gi=find_between(record.id, "ref|","|")
            if "gb" in record.id:
                gi=find_between(record.id, "gb|","|") 
            if "sp" in record.id:
                pi=find_between(record.id, "sp|","|") 
                handle = Entrez.elink(dbFrom = "protein", db = "protein",id=pi)
                recordtest = str(Entrez.read(handle))
                handle.close()
                for match in re.finditer("(?<=Id': ')[0-9]+", recordtest):
                            gi=match.group()
            ref=str(record.seq)
            # fetch all genomes associated with protein
            handle = Entrez.elink(dbFrom = "protein", db = "nucleotide",id=gi)
            record3 = str(Entrez.read(handle))
            handle.close()
            listgenes=re.findall("(?<=Id': ')[0-9]+",record3)
            listgenes=listgenes[:int(len(listgenes)/2)]
            for name1 in listgenes[:2]:
                dnahandle1 = Entrez.efetch(db="nucleotide", id=name1, rettype='gbwithparts',retmode="xml")
                record4 = Entrez.read(dnahandle1)
                dnahandle1.close()
                definition=str(record4[0]["GBSeq_definition"])
                species = record4[0]["GBSeq_organism"]
                listproteins=[]
                listsequences=[]
                listproducts=[]
                # find protein of interrest in genome and search neighbouring genes for "P450"
                for match in re.finditer(ref, str(record4)):
                    proteinspan1=match.span()[0]
                    proteinspan2=int((proteinspan1))-25000
                    proteinspan3=int((proteinspan1))+27000
                    intervalrecord=str(record4)[proteinspan2:proteinspan3]                   
                    if "P450" in intervalrecord or "p450" in intervalrecord:
                        for match in re.finditer("(?<={'GBQualifier_name': 'product', 'GBQualifier_value': ')...................", intervalrecord):
                            if "P450" in match.group() or "p450" in match.group():
                                interval2= intervalrecord[match.span()[0]-1000:match.span()[0]+1000]
                                listproducts.append(match.group())
                                for match in re.finditer("(?<={'GBQualifier_name': 'translation', 'GBQualifier_value': ')\w+", interval2):
                                    listsequences.append(match.group())
                                    translation=match.group()
                                for match in re.finditer("(?<={'GBQualifier_name': 'locus_tag', 'GBQualifier_value': ')\w+", interval2):
                                    listproteins.append(match.group())
                                    locus=match.group()
                                listp450.append(SeqRecord(Seq(translation),id=locus))
                                listferredoxins.append(record)
                        res = [] 
                        [res.append(x) for x in listproteins if x not in res]
                        
                        #safe results
                        new_row={'Protein Reference':gi,'Acessiongenome':name1,'Species':species,"Definition":definition,"proteins":res,"blasts":listsequences,"products":listproducts}
                        tableofproteins = tableofproteins.append(new_row, ignore_index=True)
                        
                        #safe every 100 entries
                        if countsafe% 100 == 0:
                            tableofproteins.to_csv(filename_output, index=False) 
                            SeqIO.write(listferredoxins, outputferre, 'fasta')
                            SeqIO.write(listp450, outputp450, 'fasta')
    except: pass
      
#final saving of files
tableofproteins.to_csv(filename_output, index=False) 
SeqIO.write(listferredoxins, outputferre, 'fasta')
SeqIO.write(listp450, outputp450, 'fasta')

# Get Ferredoxins, that are not associated with p450s

In [3]:
import re
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
from Bio.SeqRecord import SeqRecord

# fill in correct input/output file names
filename_input="Fasta/ferredoxinncbi.fasta"
filename_output=filename_input+"infoeinzel2.csv"
outputferre=filename_input+"ferreeinzel2.fasta"
outputp450=filename_input+"p450einzel2.fasta"
tableofproteins=pd.DataFrame(columns=['Protein Reference','Acessiongenome',"p450 product","p450 accessions","p450 loci","p450 translations","ferredoxin product","ferredoxin accession","ferredoxin loci","ferredoxin translation"])
Entrez.email = "friederike@biermann-erfurt.de"
listfasta=[]
liststrains=[]

a=700
b=10000
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def find_infrontof( s, last ):
    try:
        start = 0
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
def find_after( s, first ):
    try:
        start =s.index( first ) + len( first )
        end = len(s)
        return s[start:end]
    except ValueError:
        return ""
def list_diff(my_list1, my_list2):
       return (list(set(my_list1) - set(my_list2)))

listp450=[]
listferredoxins=[]
countsafe=0
#actually finding in fastas
for record in SeqIO.parse(filename_input, "fasta"):
    
    try:
            countsafe=countsafe+1
            if "ref" in record.id:
                gi=find_between(record.id, "ref|","|")
            if "gb" in record.id:
                gi=find_between(record.id, "gb|","|") 
            if "sp" in record.id:
                pi=find_between(record.id, "sp|","|") 
                handle = Entrez.elink(dbFrom = "protein", db = "protein",id=pi)
                recordtest = str(Entrez.read(handle))
                handle.close()
                for match in re.finditer("(?<=Id': ')[0-9]+", recordtest):
                            gi=match.group()
            ref=str(record.seq)
            handle = Entrez.elink(dbFrom = "protein", db = "nucleotide",id=gi)
            record3 = str(Entrez.read(handle))
            handle.close()
            listgenes=re.findall("(?<=Id': ')[0-9]+",record3)
            listgenes=listgenes[:int(len(listgenes)/2)]
            for name1 in listgenes[:2]:
                dnahandle1 = Bio.Entrez.efetch(db="nucleotide", id=name1, rettype='gbwithparts',retmode="text")
                record4 = SeqIO.read(dnahandle1, "genbank")
                dnahandle1.close()
                listp450proteins=[]
                listp450sequences=[]
                listp450products=[]
                listferredoxinproteins=[]
                listferredoxinsequences=[]
                listferredoxinproducts=[]
                listp450loci=[]
                listferredoxinloci=[]
                intervalrecord=str(record4)
                if len(record4.seq)>2000000:
                    for feature in record4.features:
                        if feature.type=="CDS":
                            if "ferredoxin" in feature.qualifiers['product'][0]:
                                try:
                                    if len(feature.qualifiers['translation'][0])<150:
                                        product=feature.qualifiers['product'][0]
                                        pid=feature.qualifiers['protein_id'][0]
                                        locus=feature.location
                                        transl=feature.qualifiers['translation'][0]
                                        listferredoxinsequences.append(transl)
                                        listferredoxinproducts.append(product)
                                        listferredoxinloci.append(locus)
                                        listferredoxinproteins.append(pid)
                                except: pass
                            if "P450" in feature.qualifiers['product'][0]:
                                try:
                                    #print(feature.qualifiers)
                                    product=feature.qualifiers['product'][0]
                                    pid=feature.qualifiers['protein_id'][0]
                                    transl=feature.qualifiers['translation'][0]
                                    locus=feature.location
                                    listp450proteins.append(pid)
                                    listp450sequences.append(transl)
                                    listp450products.append(product)
                                    listp450loci.append(locus)
                                except: pass
                    # compare loci list of ferredoxins/p450, searche for isolated ones
                    templistferredoxins=[]
                    templistp450=[]
                    listp450locin=[]
                    for locus in listp450loci:
                         listp450locin.append(locus.start)
                    listferredoxinlocin=[]
                    for locus in listferredoxinloci:
                            listferredoxinlocin.append(locus.start)
                    if len(listferredoxinloci)>1:
                        for ferredoxin in listferredoxinloci:
                            c=int(ferredoxin.start)+b
                            d=int(ferredoxin.end)-b
                            for p450 in listp450loci:
                                if c>int(p450.start)>d or c>int(p450.end)>d :
                                    print ("zu nah")
                                    if p450 not in templistp450:
                                        templistp450.append(p450.start)
                                    if ferredoxin not in templistferredoxins:
                                        templistferredoxins.append(ferredoxin.start)
                        listnewp450=list_diff(listp450locin,templistp450)
                        listnewferredoxin=list_diff(listferredoxinlocin,templistferredoxins)
                    else:
                        listnewp450=listp450locin
                        listnewferredoxin=listferredoxinlocin
                    if len(listnewferredoxin)==1 and len(listnewp450)>0:
                        countsafe=countsafe+1
                        for index, value in enumerate(listferredoxinlocin):
                                if value in listnewferredoxin:
                                    translationf=listferredoxinsequences[index]
                                    locusf=listferredoxinloci[index]
                                    IDf=listferredoxinproteins[index]
                                    productf=listferredoxinproducts[index]
                                    listferredoxins.append(SeqRecord(Seq(translationf),id=IDf))
                        for index, value in enumerate(listp450locin):
                                if value in listnewp450:
                                    translation=listp450sequences[index]
                                    locus=listp450loci[index]
                                    ID=listp450proteins[index]
                                    product=listp450products[index]
                                    listp450.append(SeqRecord(Seq(translation),id=ID))
                                    new_row={'Protein Reference':gi,'Acessiongenome':name1,"p450 product":product,"p450 accessions":ID,"p450 loci":str(locus),"p450 translations":translation,"ferredoxin product":productf,"ferredoxin accession":IDf,"ferredoxin loci":str(locusf),"ferredoxin translation":translationf}
                                    tableofproteins = tableofproteins.append(new_row, ignore_index=True)
                                if countsafe% 100 == 0:
                                    tableofproteins.to_csv(filename_output, index=False) 
                                    SeqIO.write(listferredoxins, outputferre, 'fasta')
                                    SeqIO.write(listp450, outputp450, 'fasta')
              
    except: pass
    
tableofproteins.to_csv(filename_output, index=False) 
SeqIO.write(listferredoxins, outputferre, 'fasta')
SeqIO.write(listp450, outputp450, 'fasta')

WP_207629503.1
WP_207628008.1
1821604050
8112195
a
zu nah
[ExactPosition(252359), ExactPosition(7857552), ExactPosition(3048145), ExactPosition(722390), ExactPosition(7833625), ExactPosition(7765659), ExactPosition(7256412), ExactPosition(253596), ExactPosition(7808414), ExactPosition(1139423), ExactPosition(7809634), ExactPosition(481188), ExactPosition(5427581), ExactPosition(5426343), ExactPosition(5158187), ExactPosition(4151100), ExactPosition(6829565)] [ExactPosition(4397442), ExactPosition(1714693), ExactPosition(6514483), ExactPosition(7556822), ExactPosition(6806782)]
WP_207623627.1
WP_207619503.1
1823026339
81237
WP_207598691.1
2007941048
357176
WP_207598691.1
2007941048
357176
WP_207598691.1
2007941048
357176
WP_207576506.1
1184616905
143188
WP_207566703.1
WP_207565566.1
2007832697
466817
WP_207564100.1
2007832696


KeyboardInterrupt: 